In [1]:
import os
import json
import tqdm
import argparse

from src.util import *
from src.train import *
from src.evaluate import *

In [2]:
parser = argparse.ArgumentParser(description="TestLoop",
                                        formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--loop", default="stroll", choices=["stroll", "test"], type=str, dest="loop")
parser.add_argument("--lr", default=2e-4, type=float, dest="lr")
parser.add_argument("--batch_size", default=1, type=int, dest="batch_size")
parser.add_argument("--train_continue", default="on", choices=["on", "off"], type=str, dest="train_continue")
parser.add_argument("--num_epoch", default=100, type=int, dest="num_epoch")
parser.add_argument("--task", default="pose estimation", choices=["pose estimation"], type=str, dest="task")
parser.add_argument("--ny", default=256, type=int, dest="ny")
parser.add_argument("--nx", default=256, type=int, dest="nx")
parser.add_argument("--nch", default=3, type=int, dest="nch")
parser.add_argument("--nker", default=64, type=int, dest="nker")
parser.add_argument("--norm", default='inorm', type=str, dest="norm")
parser.add_argument("--network", default="PoseResNet", choices=["PoseResNet"], type=str, dest="network")
parser.add_argument("--resnet_depth", default=50, choices=[18, 34, 50, 101, 152], type=int, dest="resnet_depth")
parser.add_argument("--joint_weight", default=False, type=bool, dest="joint_weight")
parser.add_argument("--cuda", default="cuda", choices=["cuda", "cuda:0", "cuda:1"], type=str, dest="cuda")
parser.add_argument("--spec", default="all", type=str, dest="spec")

args = parser.parse_args(args=[])
vars = vars(args)

In [3]:
cwd = os.getcwd()
datasets_dir = os.path.join(cwd, "datasets")
test_report_dir = os.path.join(cwd, "test_results")
test_design_list = os.listdir(datasets_dir)

In [ ]:
eval_results = {}
for idx_d, design in enumerate(tqdm.tqdm(test_design_list)):
    setups_dir = os.path.join(datasets_dir, design)
    reports_dir = os.path.join(test_report_dir, design)

    setups_list = os.listdir(setups_dir)
    for idx_s, setup in enumerate(setups_list):
        # Set arguments for evaluation
        vars["mode"] = "test"

        train_data_dir = os.path.join(setups_dir, setup)
        vars["data_dir"] = train_data_dir

        with open(os.path.join(train_data_dir, "train", "labels", "mpii_style.json"), "r", encoding="utf-8") as fread:
            labels_dict = json.load(fread)
            num_mark = len(labels_dict[0]["joints_vis"])
        vars["num_mark"] = num_mark

        args.ckpt_dir = os.path.join(reports_dir, setup, "checkpoint")

        # Evaluate
        evals = evaluate(args=args)
        eval_results["%s" % design+"-"+setup] = evals

  0%|                                                                                                                                                                   | 0/2 [00:00<?, ?it/s]C:\Users\come1\anaconda3\envs\pytorch38\lib\site-packages\torch\nn\functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
 50%|█████████████████████████████████████████████████████████████████████████████                                                                             | 1/2 [02:02<02:02, 123.00s/it]

In [ ]:
save_dir = os.path.join(test_report_dir, "evaluation.json")
with open(save_dir, "w", encoding = "UTF-8-SIG") as file:
    json.dump(eval_results, file, ensure_ascii=False)